# Algoritmo que permite resolver problemas QUBO densos empleando Tensor Network

En este notebook vamos a crear e implementar un algoritmo en tensor networks basado en el método de las señales y evolución en tiempo imaginario para resolver problemas QUBO. Utiliza un método de eliminación de capas de evolución para reducir la complejidad computacional.

Versiones:

- alpha 0: Implementación básica.
- alpha 1: Añade el método de eliminación de capas.
- alpha 2: Versión que va comprimiendo el MPS de derecha a izquierda
- alpha 3: version que aplica el problema QUBO a los vecinos cercanos, la idea de esta versión es aplicar matrices recursivamente, además de intentar aprovechar los cálculos intermedios

In [3]:
# Librerias
import numpy as np
import torch
from quantum_sim.main.general_functions import dinariy_list
from quantum_sim.TensorNetwork.QUBO.qubo_core.qubo_solvers import qubo_dimod_solver, recocido_simulado, qubo_solver_rs, random_qubo_solver
from quantum_sim.TensorNetwork.QUBO.qubo_core.qubo_auxiliar_functions import matrix_QUBO_to_dict, evaluar_qubo, generar_matriz_qubo
from quantum_sim.main.guardar_experimentos import plot_function



%load_ext autoreload
%autoreload 2

---
# Funciones de la tensor network

## Definimos los nodos de la Tensor Network

In [4]:
def node_0(Q_matrix_0: float, dits: int, n_neighbors:int):
    tensor = np.zeros((dits, dits**(n_neighbors -1) ))
    din_list = dinariy_list(dits**(n_neighbors -1), dits)
    for dit in range(dits): 
        for index, element in enumerate(din_list):
            if str(dit) == element[-1]:
                    tensor[dit][index] = 1
        if dit > 0:
            tensor[dit] = tensor[dit] * dit * Q_matrix_0
    return tensor

def node_intermediate(Q_matrix_column_input: np.array, dits: int):
    n_neighbors = (len(Q_matrix_column_input)-1)
    tensor_size = int(dits**(n_neighbors))
    tensor = np.zeros((tensor_size, tensor_size))
    din_list = dinariy_list(tensor_size, dits)
    for row, element in enumerate(din_list):
        for column, element_2 in enumerate(din_list):
            if element[1:] == element_2[:-1]:
                if element_2[-1] != '0':
                    tensor[row, column] = Q_matrix_column_input[-1] * int(element_2[-1])
                    for index, digit in enumerate(element):
                        if digit != '0':
                            tensor[row, column] *= Q_matrix_column_input[index]* int(digit)
                else:
                    tensor[row, column] = 1   
           
    return tensor
                    

def last_tensor(Q_matrix_column_input: np.array, dits: int):
    n_neighbors = (len(Q_matrix_column_input)-1)
    tensor_size = int(dits**(n_neighbors))
    tensor = np.ones((tensor_size))
    din_list = dinariy_list(tensor_size, dits)
    for index, element in enumerate(din_list):
        if element[-1] != "0":
            tensor[index] = Q_matrix_column_input[-1] * int(element[-1])
            for digit in element:
                if digit != '0':
                    tensor[index] *= Q_matrix_column_input[int(digit)] * int(digit)

    return tensor





## Generación de la Tensor Network

In [5]:
def tensor_network_generator(Q_matrix:np.array, dits:float, n_neighbors:int):
    """   
    Args:

    Return:

    """
    
    n_variables  = len(Q_matrix[0])
    intermediate_tensors = []
    # generation of the first node
    tensor = node_0(Q_matrix[0][0], dits , n_neighbors)
    intermediate_tensors.append(tensor)
    
    # Generation of the intermediate nodes
    for variable in range(1, n_variables-1):

        
        Q_matrix_row_input = Q_matrix[variable][max(0, variable-2):variable+1]
        while len(Q_matrix_row_input) < n_neighbors:
            Q_matrix_row_input = np.insert(Q_matrix_row_input, 0, 0)

        tensor = node_intermediate(Q_matrix_row_input, dits)
        intermediate_tensors.append(tensor)
    Q_matrix_row_input = Q_matrix[-1][-n_neighbors:]

    tensor = last_tensor(Q_matrix_row_input, dits)
    intermediate_tensors.append(tensor)

    return  intermediate_tensors

## Conexión y contracción de toda la Tensor Network

In [6]:
def tensor_network_contraction(tensor_list: list):
    n_tensores = len(tensor_list)
    intermediate_tensors = []
    tensor = tensor_list[-1]
    intermediate_tensors.append(tensor)
    for index in range(n_tensores -1, 0, -1):
        current_tensor = tensor_list[index -1]
        tensor = current_tensor @ tensor
        intermediate_tensors.append(tensor)
    intermediate_tensors.reverse()
    return tensor, intermediate_tensors
    

---
# Función general

Esta es la función que se encarga del proceso general. Se encarga del proceso de minimización resolviendo iterativamente cada una de las variables. Su proceso consiste en la creación de la tensor network, su contracción y la determinación de la variable a partir del vector resultante.

In [7]:
from unittest import result

from quantum_sim.main.general_functions import base_a_decimal


def qubo_solver(Q_matrix:np.array, tau:float, dits: int, n_neighbors: int)->np.array:
    n_variables = Q_matrix.shape[0]
    Q_matrix = np.exp(-tau*Q_matrix)
    print(Q_matrix)
    solution = np.zeros(n_variables, dtype = int)
    tensor_network = tensor_network_generator(Q_matrix, dits, n_neighbors)
    for t in tensor_network:
        print("tensor",t.shape)
        print(t)
    result_contraction, intermediate_tensors = tensor_network_contraction(tensor_network)
    aux_solution = '0'*(n_neighbors-1)
    solution[0] = np.argmax(abs(result_contraction))
    aux_solution = aux_solution[1:] + str(solution[0])
    for node in range(1, n_variables-1):
        index_result = base_a_decimal(aux_solution, dits)
        new_tensor = tensor_network[node][index_result]
        print("newt")
        print(new_tensor)


        

    


    return solution


---
# Pruebas

In [8]:
n_variables = 5
n_vecinos = 3
dits = 2
tau = 1
5
np.random.seed(78)
# Generamos el caso
Q_matrix = generar_matriz_qubo(n_variables, n_vecinos)
Q_matrix/= np.linalg.norm(Q_matrix)
Q_matrix_dict = matrix_QUBO_to_dict(Q_matrix)


# Inicial RS
x_inicial = np.random.randint(2, size=n_variables)

# TN
solution = qubo_solver(Q_matrix, tau, dits, n_vecinos)
print('Solution TN:     ', str(solution))
print('Coste de TN:     ', evaluar_qubo(Q_matrix, solution))

# RS
solution = recocido_simulado(Q_matrix, x_inicial, 10.0, 0.99, int(1e4))
#print('Solution RS:     ', str(solution))
print('Coste de RS:     ', evaluar_qubo(Q_matrix, solution))

# RS iterativo
solution = qubo_solver_rs(Q_matrix, number_layers)
#print('Solution RS iter: ', str(solution))
print('Coste de RS iter: ', evaluar_qubo(Q_matrix, solution))

# Dimod

solution_dimod = qubo_dimod_solver(Q_matrix_dict, "neal")
print('Coste de dimod iter: ', evaluar_qubo(Q_matrix, solution_dimod))

best_c = random_qubo_solver(Q_matrix)
print('Coste de Random: ', evaluar_qubo(Q_matrix, best_c))


[[0.64595725 1.         1.         1.         1.        ]
 [1.05563962 0.72586799 1.         1.         1.        ]
 [0.86474765 0.66551642 0.79380644 1.         1.        ]
 [1.         1.2219109  0.69679031 0.69037298 1.        ]
 [1.         1.         1.11243437 1.40197339 0.8416946 ]]
tensor (2, 4)
[[1.         0.         1.         0.        ]
 [0.         0.64595725 0.         0.64595725]]
tensor (4, 4)
[[1.         0.72586799 0.         0.        ]
 [0.         0.         1.         0.766255  ]
 [1.         0.         0.         0.        ]
 [0.         0.         1.         0.        ]]
tensor (4, 4)
[[1.         0.79380644 0.         0.        ]
 [0.         0.         1.         0.52829122]
 [1.         0.68644226 0.         0.        ]
 [0.         0.         1.         0.45683859]]
tensor (4, 4)
[[1.         0.69037298 0.         0.        ]
 [0.         0.         1.         0.48104521]
 [1.         0.84357428 0.         0.        ]
 [0.         0.         1.         0.58

NameError: name 'number_layers' is not defined